In [ ]:
function impute_genotypes(geno,ped,mme,Ai_nn,Ai_ng,big_memory=false)
    num_pedn = size(Ai_nn,1)
    #reorder genotypes to get Mg with the same order as Ai_gg
    Z  = mkmat_incidence_factor(ped.IDs[(num_pedn+1):end],geno.obsID)
    Mg = Z*geno.genotypes
    #**********************************************************
    #impute genotypes for non-genotyped inds
    #**********************************************************
    #two approached are available with similar speed and memory
    #appraoch 1
    #Ai_nn*Mn = -Ai_ng*Mg => [Ai_nn\(-Ai_ng*Mg)
    #                         Mg];
    #
    #appraoch 2
    #[Ai_nn 0 [Mn    [-Ai_ng
    #    0 I]* Mg] =       I]*Mg
    ngeno   = size(Ai_ng,2)
    block12 = spzeros(num_pedn, ngeno)
    block21 = spzeros(ngeno,num_pedn)
    block22 = sparse(I, ngeno, ngeno)
    lhs     = [Ai_nn block12
               block21 block22];
    rhs     = [-Ai_ng
               block22];
    #genotypes are imputed for all individuals in pedigree and aligned to
    #phenotyped individuals by genotype chunks to avoid memory issues.
    Z              = mkmat_incidence_factor(mme.obsID,ped.IDs)
    if big_memory == false
        nmarkers       = size(Mg,2)
        Mpheno         = zeros(length(mme.obsID),nmarkers)
        markerperchunk = 1000
        if nmarkers%markerperchunk == 0
            nchunks = div(nmarkers,markerperchunk)
        else
            nchunks = div(nmarkers,markerperchunk)+1
        end
        @showprogress "imputing genotypes for non-genotyped individuals" for i=1:nchunks
            if i != nchunks
                myrange = (1+(i-1)*markerperchunk):(i*markerperchunk)
            else
                myrange = (1+(i-1)*markerperchunk):nmarkers
            end
            Mped_chunk        = lhs\(rhs*Mg[:,myrange])
            Mpheno[:,myrange] = Z*Mped_chunk
        end
    else
         Mpheno = Z*(lhs\(rhs*Mg))
    end
    mme.M.genotypes = Mpheno
    mme.M.obsID     = mme.obsID
    mme.M.nObs      = length(mme.M.obsID)
    GC.gc()
end